In [1]:
import sys
sys.path.append('..')

import os
import torch
from tqdm import tqdm
from copy import deepcopy
from torch.utils.tensorboard import SummaryWriter

from src.utils.get_model_and_data import get_model_and_data
from src.utils.collate_fn_coco import collate_fn_coco

In [2]:
# checkpoint_path = '../checkpoint/boxclip-finetune-coslr'
checkpoint_path = '../checkpoint/boxclip-one-cap-0806/'
checkpoint_name = 'checkpoint-epoch{}.pth.tar'

In [4]:
parameters = {
    'device': 'cuda',
    'num_attentionLayer': 4
}
model, datasets = get_model_and_data(parameters)

loading annotations into memory...
Done (t=16.70s)
creating index...
index created!
loading annotations into memory...
Done (t=0.99s)
creating index...
index created!
train set scale: 21391
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
val set scale: 925


KeyError: 'num_attentionLayer'

In [ ]:
def eval_loss(dataloader, model, device):
    dict_loss = {}
    model.eval()
    with torch.no_grad():
        for i, batch in enumerate(tqdm(dataloader)):
            for k, v in batch.items():
                if torch.is_tensor(batch[k]): batch[k] = batch[k].to(device)
                
            batch.update(model(batch))
            _, losses = model.compute_loss(batch)
            if i == 0:
                dict_loss = deepcopy(losses)
            else:
                for k in dict_loss.keys():
                    dict_loss[k] += losses[k]
        for k in dict_loss.keys():
            dict_loss[k] /= len(dataloader)
    
    dict_loss = {'val_'+k: v for k, v in dict_loss.items()}
    return dict_loss

In [ ]:
# val_losses = eval_loss(val_loader, model, 'cuda')
iter_per_eps = len(datasets['train']) // 64
if iter_per_eps * 64 < len(datasets['train']): iter_per_eps += 1
print(f'iter per epoch: {iter_per_eps}')

writer = SummaryWriter(log_dir=checkpoint_path)
val_loader = torch.utils.data.DataLoader(datasets['val'], batch_size=20, collate_fn=collate_fn_coco)

for i in range(0, 105, 5):
    try:
        checkpoint = torch.load(os.path.join(checkpoint_path, checkpoint_name.format(i)))
    except:
        print(f'{checkpoint_name.format(i)} not found.')
        continue
        
    model.load_state_dict(checkpoint['model'])
    print(f'checkpoint {checkpoint_name.format(i)} loaded.')
    
    val_losses = eval_loss(val_loader, model, 'cuda')
#     print(val_losses)
#     assert False
    writer.add_scalars("Loss/Iters", val_losses, i * iter_per_eps)

iter per epoch: 335
checkpoint-epoch0.pth.tar not found.
checkpoint checkpoint-epoch5.pth.tar loaded.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:40<00:00,  1.16it/s]


checkpoint checkpoint-epoch10.pth.tar loaded.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:27<00:00,  1.73it/s]


checkpoint checkpoint-epoch15.pth.tar loaded.


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:44<00:00,  1.07it/s]


checkpoint checkpoint-epoch75.pth.tar loaded.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 47/47 [00:28<00:00,  1.64it/s]


checkpoint checkpoint-epoch80.pth.tar loaded.


 68%|███████████████████████████████████████████████████████████████████████████▌                                   | 32/47 [00:22<00:09,  1.56it/s]